# Function to compare manual segmentation

This notebook goes through folders of manually segmented images and compares the results.  

Currently the folders are set up like this:

```
20230722-Human_Segmentation
    a_walk_in_the_park
        Maryam_GT - Maryam Berijanian.png
        a_walk_in_the_park copy_GT - Katie Reagan.jpg
        a_walk_in_the_park_GT - Catalina Diaz.jpg
        a_walk_in_the_park_GT - Dirk Colbry.png
        a_walk_in_the_park_Nathan_Haut_GT - Nathaniel haut.jpg
        a_walk_in_the_park_navya_GT - navya bhardwaj.jpg
    Mushrooms_GT
        Maryam_GT - Maryam Berijanian(1).png
        Mushrooms_GT - Dirk Colbry.png
        Mushrooms_GT - Katie Reagan.jpeg
        Mushrooms_Nathan_Haut_GT - Nathaniel haut.jpeg
        Mushrooms_navya_GT - navya bhardwaj.jpeg
    Waterbottle
        Maryam_GT - Maryam Berijanian(2).png
        Maryam_GT - Maryam Berijanian(3).png
        Waterbottle copy_GT - Katie Reagan.png
        Waterbottle_GT - Dirk Colbry.png
        Waterbottle_Nathan_Haut_GT - Nathaniel haut.jpg
        Waterbottle_navya_GT - navya bhardwaj.jpg
        catalina_GT - Catalina Diaz.jpg
```

I also copied in the masks from the see-segment folder and deleted on of the masks that included the original RGB image as background.

In [ ]:
# Add path to the see folder in see-segment

import sys
sys.path.append('../see-segment/')

In [ ]:
import see

# Path to the folder of manually selected images

I named mine 20230722-Human_Segmentation 

In [ ]:
# Modified from code by navya
import pathlib
imagepath = pathlib.Path('20230722-Human_Segmentation/')
GTS = imagepath.glob('*')
image_folders = list(GTS)
image_folders

# Load two images and compare them
This example uses images that start with Mush

In [ ]:
from cleaner import cleaner

In [ ]:
import imageio.v3 as Image
imagepath = image_folders[0]
print(f' Testing images from {imagepath.name}')
myset = list(imagepath.glob('*'))
print(myset[0])
print(myset[1])

In [ ]:
im1 = Image.imread(myset[0])


In [ ]:
mask1 = im1[:,:,0]
clean_mask1 = mask1 > 127

In [ ]:
im2 = Image.imread(myset[1])
mask2 = im2[:,:,0]
clean_mask2 = mask2 > 127

In [ ]:
import matplotlib.pylab as plt

plt.imshow(im1)

In [ ]:
plt.imshow(im2)

# Now run fitness function

In [ ]:
import see.Segment_Fitness as Fitness

In [ ]:
output = Fitness.FitnessFunction(mask1,mask2)

In [ ]:
output

In [ ]:
output[0]

In [ ]:
clean_output = Fitness.FitnessFunction(clean_mask1,clean_mask2)
clean_output[0]

# Loop and Load/Clean up the images

In [ ]:
all_orig_images = dict()
all_orig_masks = dict()
all_clean_masks = dict()
all_names = dict()

for f, folder in enumerate(image_folders):
    images = list(folder.glob('*g'))
    names = []
    orig_images = dict()
    orig_masks = dict()
    clean_masks = dict()
    for i, im_file in enumerate(images):
        im = Image.imread(im_file)
        myname = im_file.name
        names.append(myname)
        if len(im.shape) > 2:
            mask = im[:,:,0]
        else:
            mask = im
        clean_mask = mask > im.max()/2 #Magic Cleaning code
        orig_images[myname] = im
        orig_masks[myname] = mask
        clean_masks[myname] = clean_mask
    all_names[folder.name] = names
    all_orig_images[folder.name] = orig_images
    all_orig_masks[folder.name] = orig_masks
    all_clean_masks[folder.name] = clean_masks

In [ ]:
all_names

# This code is a bit slow but works. Generates tables of before and after cleaning

In [ ]:
import numpy as np

result_tables = dict()
clean_result_tables = dict()

for folder in all_names:
    print(folder)
    values = np.zeros([len(all_names[folder]), len(all_names[folder])])
    clean_values = np.zeros([len(all_names[folder]), len(all_names[folder])])
    for i,name1 in enumerate(all_names[folder]):
        print(f'--{name1}')
        for j,name2 in enumerate(all_names[folder]):
            mask1 = all_orig_masks[folder][name1]
            mask2 = all_orig_masks[folder][name2]
            out = Fitness.FitnessFunction(mask1,mask2)
            values[i,j] = out[0]
            
            clean_mask1 = all_clean_masks[folder][name1]
            clean_mask2 = all_clean_masks[folder][name2]
            clean_out = Fitness.FitnessFunction(clean_mask1,clean_mask2)
            clean_values[i,j] = clean_out[0]
            
    result_tables[folder] = values
    clean_result_tables[folder] = clean_values    
    all_names[folder] = names

# Look at the results

In [ ]:
print(list(result_tables.keys()))

In [ ]:
key = 'a_walk_in_the_park'
key = 'Waterbottle'
#key = 'Mushrooms_GT'

In [ ]:
plt.imshow(result_tables[key])
plt.xticks(ticks=range(len(all_names[key])), labels=all_names[key], rotation='vertical')
plt.yticks(ticks=range(len(all_names[key])), labels=all_names[key])
plt.title(f"{key} Before Cleaning")
plt.colorbar()
plt.savefig(f'{key}_before_cleaning.png')


In [ ]:
plt.imshow(clean_result_tables[key])
plt.title(f"{key} After Cleaning")
plt.xticks(ticks=range(len(all_names[key])), labels=all_names[key], rotation='vertical')
plt.yticks(ticks=range(len(all_names[key])), labels=all_names[key])
plt.colorbar()
plt.savefig(f'{key}_after_cleaning.png')

In [ ]:
clean_result_tables[key]

In [ ]:
indices = np.where(clean_result_tables[key].max())
print(indices)

In [ ]:
im_files = list(all_clean_masks[key].keys())
im_files

In [ ]:
num=6
print(im_files[num])
plt.imshow(all_clean_masks[key][im_files[num]])

# Calculate Errors

In [ ]:
# Original Average
np.sum(result_tables[key])/ ((len(result_tables[key])-1)**2)

In [ ]:
# Clean Average
np.sum(clean_result_tables[key])/ ((len(clean_result_tables[key])-1)**2)

# Calculate before and after for everything and plot as a bar graph

In [ ]:
before_average = dict()
after_average = dict()

for key in result_tables:
    size = (len(result_tables[key])-1)**2
    before_average[key] = np.sum(result_tables[key])/size

    after_average[key] = np.sum(clean_result_tables[key])/size

In [ ]:
before_average

In [ ]:
after_average

In [ ]:
import numpy as np
a = np.array([[1,2,3], [4,3,1]])  # Can be of any shape
indices = np.where(a == a.max())

In [ ]:
# set width of bar
barWidth = 0.35
#fig = plt.subplots(figsize =(12, 8))

before = []
after = []
names = []

for name in before_average:
    before.append(before_average[name])
    after.append(after_average[name])
    names.append(name)

# Set position of bar on X axis
br1 = np.arange(len(before))
br2 = [x + barWidth for x in br1]
 
# Make the plot
plt.bar(br1, before, color ='r', width = barWidth,
        edgecolor ='grey', label ='Average Before Cleaning')
plt.bar(br2, after, color ='g', width = barWidth,
        edgecolor ='grey', label ='Average After Cleaning')
 
# Adding Xticks
plt.xlabel('Image', fontweight ='bold', fontsize = 15)
plt.ylabel('Difference Error (fitness)', fontweight ='bold', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(br1))], names)

plt.yscale("log") 

plt.legend(loc='upper right')
plt.savefig('before_after_averages.png')